<a href="https://colab.research.google.com/github/navabro/Image-Classifier-Dogs-vs-Cats/blob/main/dog_vs_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow tensorflow-datasets matplotlib pillow


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt



In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, tf.cast(label, tf.float32)

(train_ds, val_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True
)

train_ds = train_ds.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds = val_ds.map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=5)



In [ ]:
loss, acc = model.evaluate(val_ds)
print("Validation Accuracy:", acc)

model.save("cats_vs_dogs_model.h5")


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
from PIL import Image
import numpy as np

IMG_PATH = list(uploaded.keys())[0]
img = Image.open(IMG_PATH).convert("RGB").resize((IMG_SIZE, IMG_SIZE))
arr = np.array(img)/255.0
arr = np.expand_dims(arr, axis=0)

pred = model.predict(arr)[0][0]
print("Prediction:", "Dog 🐶" if pred>0.5 else "Cat 🐱")


In [ ]:
from google.colab import files
files.download("cats_vs_dogs_model.h5")

